# Music Recommender System using Apache Spark and Python

## Description

Create a recommender system that will recommend new musical artists to a user based on their listening history. Suggesting different songs or musical artists to a user is important to many music streaming services, such as Pandora and Spotify. In addition, this type of recommender system could also be used as a means of suggesting TV shows or movies to a user (e.g., Netflix). 

Spark and the collaborative filtering technique is used for this project. 

## Datasets

Data is from publicly avaiable song data from audioscrobbler, which can be found [here](http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html). 

The original data files are modified so that the code will run in a reasonable time on a single machine. The reduced data files have been suffixed with `_small.txt` and contains only the information relevant to the top 50 most prolific users (highest artist play counts).

The original data file `user_artist_data.txt` contained about 141,000 unique users, and 1.6 million unique artists. About 24.2 million users’ plays of artists are recorded, along with their count.

Note that when plays are scribbled, the client application submits the name of the artist being played. This name could be misspelled or nonstandard, and this may only be detected later. For example, "The Smiths", "Smiths, The", and "the smiths" may appear as distinct artist IDs in the data set, even though they clearly refer to the same artist. So, the data set includes `artist_alias.txt`, which maps artist IDs that are known misspellings or variants to the canonical ID of that artist.

The `artist_data.txt` file then provides a map from the canonical artist ID to the name of the artist.

## Necessary Package Imports

In [15]:
# make pyspark importable as a regular library
import findspark
findspark.init()

In [16]:
from pyspark.mllib.recommendation import *
from pyspark import SparkContext, SparkConf
import random
import os
from operator import *

In [17]:
#sc =SparkContext()
sc = SparkContext.getOrCreate()
sc.setLogLevel("ERROR")

## Loading data

Load the three datasets into RDDs and name them `artistData`, `artistAlias`, and `userArtistData`.

In [21]:
#read from artist alias data files
artistAlias=sc.textFile('./data/artist_alias_small.txt')
text_alias_map=artistAlias.map(lambda x: x.split('\t')).map(lambda y: (int(y[0]),int(y[1])))
dict_alias=dict(text_alias_map.collect())

#read artist data file
artistData=sc.textFile('./data/artist_data_small.txt')
text_artist_map=artistData.map(lambda x: x.split('\t')).map(lambda y: (int(y[0]),y[1]))

#correct artist id in user_artist_data file
file_user = open('./data/user_artist_data_small.txt', 'r')
#prepare file with correct artist id
if os.path.exists('./data/correct_user_artist_data_small.txt'):
    os.remove('./data/correct_user_artist_data_small.txt')
file_output = open('./data/correct_user_artist_data_small.txt', "a+")
    
for line_file_user in file_user:
    line_file_user_cell=line_file_user.strip('\n').split('\t')
    cell_str=line_file_user_cell[0].split(' ')
    #check if artist id in alias dict. if so, replace it with correct artist id
    if int(cell_str[1]) in dict_alias.keys():
        cell_str[1]=dict_alias[int(cell_str[1])]
    line= str(cell_str[0])+','+	str(cell_str[1])+','+str(cell_str[2])
    file_output.write("%s\n" %(line))
file_output.close()
file_user.close()

#correct user_artist_data file now in file 'correct_user_artist_data_small.txt'
userArtistData=sc.textFile('./data/correct_user_artist_data_small.txt')

## Data Exploration

Find the users' total play counts. Find the three users with the highest number of total play counts (sum of all counters) and print the user ID, the total play count, and the mean play count (average number of times a user played an artist). Your output should look as follows:
```
User 1059637 has a total play count of 674412 and a mean play count of 1878.
User 2064012 has a total play count of 548427 and a mean play count of 9455.
User 2069337 has a total play count of 393515 and a mean play count of 1519.
```


In [22]:
text_user_map=userArtistData.map(lambda x: x.split(',')).map(lambda y: (int(y[0]),int(y[1]), int(y[2])))
#compute the sum of plays
text_user_pairs=text_user_map.map(lambda s: (s[0], s[2]))
text_user_counts=text_user_pairs.reduceByKey(lambda a,b:a+b)
#compute mean
text_user_mean=text_user_pairs.groupByKey().mapValues(lambda x: sum(x)/len(x))
#order by sum
text_user_ordered=text_user_counts.join(text_user_mean).sortBy(lambda a:a[1],ascending=False)
#take top 3
text_user_ordered_top_3=text_user_ordered.take(3)
for cell in text_user_ordered_top_3:
    print 'User', cell[0], 'has a total play count of', cell[1][0], 'and a mean play count of', cell[1][1], '.'

User 1059637 has a total play count of 674412 and a mean play count of 1878 .
User 2064012 has a total play count of 548427 and a mean play count of 9455 .
User 2069337 has a total play count of 393515 and a mean play count of 1519 .


####  Splitting Data for Testing

Use the [randomSplit](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) function to divide the data (`userArtistData`) into:
* A training set, `trainData`, that will be used to train the model. This set should constitute 40% of the data.
* A validation set, `validationData`, used to perform parameter tuning. This set should constitute 40% of the data.
* A test set, `testData`, used for a final evaluation of the model. This set should constitute 20% of the data.

Use a random seed value of 13. Since these datasets will be repeatedly used you will probably want to persist them in memory using the [cache](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.cache) function.

In addition, print out the first 3 elements of each set as well as their sizes; if you created these sets correctly, your output should look as follows:
```
[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000113, 5)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000112, 423)]
[(1059637, 1000094, 1), (1059637, 1000130, 19129), (1059637, 1000139, 4)]
19817
19633
10031
```

In [23]:
#split file into 3 data sets
trainData, validationData, testData = text_user_map.randomSplit([40, 40, 20], seed=13)
trainData.cache()
validationData.cache()
testData.cache()
print trainData.take(3)
print validationData.take(3)
print testData.take(3)
print trainData.count()
print validationData.count()
print testData.count()


[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000113, 5)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000112, 423)]
[(1059637, 1000094, 1), (1059637, 1000130, 19129), (1059637, 1000139, 4)]
19818
19632
10031


## The Recommender Model

For this project, we will train the model with implicit feedback. You can read more information about this from the collaborative filtering page: [http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html). The [functionn you will be using](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS.trainImplicit) has a few tunable parameters that will affect how the model is built. Therefore, to get the best model, we will do a small parameter sweep and choose the model that performs the best on the validation set

Therefore, we must first devise a way to evaluate models. Once we have a method for evaluation, we can run a parameter sweep, evaluate each combination of parameters on the validation data, and choose the optimal set of parameters. The parameters then can be used to make predictions on the test data.

### Model Evaluation

Although there may be several ways to evaluate a model, we will use a simple method here. Suppose we have a model and some data containing a user and the artists we know the user listened to. We can use this data to validate the model. Have the model predict the user's preference for all artists in the dataset, rank them, sort them according to the rank, and select the top X artists for that user, with X being the number of artists that the user actually listened to (this may be different for each user). Then, the fraction of overlap between the top X predictions of the model and the X real artists that the user played can be calculated. This process can be repeated for all users and an average value can be returned.

For example, suppose a model predicted [1,2,4,8] as the top X=4 artists for a user. Suppose, that user actually listened to the artists [1,3,7,8]. Then, for this user, the model would have a score of 2/4=0.5. To get the overall score, this would be performed for all users, with the average returned.

**NOTE: you will be using this function for both the validation data and the test data. When using the validation data, predict on all artists not in the user's training or test set. When using this function on the test data, predict on all artists not in the user's training or validation set. The number of artists you are predicting for each user will be different.**

For example, suppose all the artists in the entire dataset are [1,2,3,4,5,6,7,8,9]. In the training set user A listened to [1,2], in the validation set [3] and the test set [4,5]. Then, when using the validation data, X=1 and you predict on [3,6,7,8,9]. When using the test data, X=2 and you predict on [4,5,6,7,8,9]. This ensures you aren't penalizing the model for predicting correct answers that aren't in the specific dataset that is being tested.

Name your function `modelEval` and have it take a model (the output of ALS.trainImplicit), a dataset, and a flag as input. For parameter tuning, the dataset parameter should be set to the validation data (`validationData`). After parameter tuning, the model can be evaluated on the test data (`testData`). The flag should indicate if it's test data or validation data as you will need to do something different for each (see bold above).

In [24]:
#evaluation module
def modelEval(model, dataset,f_test_valid):
    #all artist list
    all_artist_list = set(text_user_map.map(lambda x: x[1]).collect())
    #all user list in data set (validation or testing)
    all_user_list = set(dataset.map(lambda x: x[0]).collect())
    #dictionary of user and artists list in input dataset. Key by user id
    artists_per_user_in_dataset = dict(dataset.map(lambda x: (x[0],x[1])).groupByKey().mapValues(set).collect())
    #dictionary of user and artists list in train dataset. Key by user id
    artists_per_user_in_train = dict(trainData.map(lambda x: (x[0],x[1])).groupByKey().mapValues(set).collect())
    #dictionary of user and artists list in test dataset. Key by user id
    artists_per_user_in_test = dict(testData.map(lambda x: (x[0],x[1])).groupByKey().mapValues(set).collect())
    #dictionary of user and artists list in validation dataset. Key by user id
    artists_per_user_in_valid = dict(validationData.map(lambda x: (x[0],x[1])).groupByKey().mapValues(set).collect())
    hit_ratio = []
    #calculate hit_ratio for each user
    for user in all_user_list: 
        if f_test_valid=='valid':
            artists_for_prediction = all_artist_list - artists_per_user_in_train[user]-artists_per_user_in_test[user]  	
        if f_test_valid=='test':
            artists_for_prediction = all_artist_list - artists_per_user_in_train[user]-artists_per_user_in_valid[user]  
        #list of artists the user listened
        artists_listened = artists_per_user_in_dataset[user]
        X = len(artists_listened)
        user_artists_for_prediction = map(lambda x: (user, x),artists_for_prediction)
        #get artists (referred as products) predictions for the current user
        predictionsArtists = model.predictAll(sc.parallelize(user_artists_for_prediction)).sortBy(ascending=False, keyfunc = lambda x: x.rating).map(lambda x:x.product).take(X)
        
        hit_ratio.append((float(len(set(predictionsArtists).intersection(artists_listened))) / X))
        #print "top X:",X
    sum_score=0
    avg_score=0 
    #print 'total users in data set',len(all_user_list)   
    for i in range (len(hit_ratio)):
        sum_score=sum_score+hit_ratio[i]
        #print sum_score
    avg_score=sum_score/len(all_user_list)
    return avg_score


### Model Construction

Now we can build the best model possibly using the validation set of data and the `modelEval` function. Although, there are a few parameters we could optimize, for the sake of time, we will just try a few different values for the [rank parameter](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html#collaborative-filtering) (leave everything else at its default value, **except make `seed`=345**). Loop through the values [2, 10, 20] and figure out which one produces the highest scored based on your model evaluation function.

Note: this procedure may take several minutes to run.

For each rank value, print out the output of the `modelEval` function for that model. Your output should look as follows:
```
The model score for rank 2 is 0.097347
The model score for rank 10 is 0.100320
The model score for rank 20 is 0.085303
```

In [25]:
#determine best rank parameter value 
ranks = [2, 10, 20]
rk_select=0
max_score=0
for rk in ranks:
    model = ALS.trainImplicit(trainData, rk, seed=345)
    score= modelEval(model, validationData,'valid')
    print 'The model score for rank ',rk, ' is ', score
    if score>max_score:
        rk_select=rk
        max_score=score
print 'Selected rank= ', rk_select

The model score for rank  2  is  0.0878821632731
The model score for rank  10  is  0.104440929322
The model score for rank  20  is  0.089668567031
Selected rank=  10


Now, using the bestModel, we will check the results over the test data. Your result should be ~`0.065`.

In [26]:
#best model using rank=10
bestModel = ALS.trainImplicit(trainData, rank=rk_select, seed=345)
modelEval(bestModel, testData, 'test')
print 'The best model score using test data set is ', modelEval(bestModel, testData, 'test')

The best model score using test data set is  0.0658012108774


## Trying Some Artist Recommendations
Using the best model above, predict the top 5 artists for user `1059637` using the [recommendProducts](http://spark.apache.org/docs/1.5.2/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.recommendProducts) function. Map the results (integer IDs) into the real artist name using `artistAlias`. Print the results. The output should look as follows:
```
Artist 0: blink-182
Artist 1: Elliott Smith
Artist 2: Taking Back Sunday
Artist 3: Incubus
Artist 4: Death Cab for Cutie
```

In [27]:
#output top recommendation
ratingObj = bestModel.recommendProducts(1059637, 5)
recomArtists = map(lambda x : x.product, ratingObj)
artistDict = dict(text_artist_map.collect())
i=0
for artist in recomArtists:
    print "Artist "+ str(i)+ ":",artistDict[artist]
    i=i+1
   

Artist 0: Taking Back Sunday
Artist 1: Elliott Smith
Artist 2: Marilyn Manson
Artist 3: Brand New
Artist 4: Death Cab for Cutie


In [28]:
sc.stop()